# Transformation of Santander dataset into Articles, Customers, Transaction (ACT) format

In [ ]:
%load_ext kedro.extras.extensions.ipython

In [ ]:
%reload_kedro

In [ ]:
from typing import Iterator, Tuple
import re

import pandas as pd
import numpy as np
from kedro.extras.datasets.pandas import CSVDataSet
from kedro.io.core import get_filepath_str

import gid_ml_framework.pipelines.santander_preprocessing.nodes
from gid_ml_framework.extras.datasets.chunks_dataset import (
 _load,
 _concat_chunks,
)

pd.options.mode.chained_assignment = None
pd.set_option('expand_frame_repr', True)
pd.set_option("display.max_rows", 999)
pd.set_option('max_colwidth', 100)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.expand_frame_repr', False)

## Final dataframes check

In [ ]:
santander_articles_path = "santander_to_act_main.santander_articles"
santander_customers_path = "santander_to_act_main.santander_customers"
santander_trainsactions_train_path = "santander_to_act_main.santander_transactions_train"
santander_trainsactions_val_path = "santander_to_act_main.santander_transactions_val"

In [ ]:
santander_customers = _concat_chunks(context
                                     .catalog.load(santander_customers_path))
santander_articles = _concat_chunks(context
                                    .catalog.load(santander_articles_path))
santander_transactions_train = _concat_chunks(context
                                              .catalog
                                              .load(santander_trainsactions_train_path))
santander_transactions_val = _concat_chunks(context
                                              .catalog
                                              .load(santander_trainsactions_val_path))

In [ ]:
santander_articles

In [ ]:
santander_customers

In [ ]:
santander_transactions_train

## Transactions transformations optimization

In [ ]:
train_path = "santander_train"
val_path = "santander_val"
test_path = "santander_test"

In [ ]:
train = _concat_chunks(context.catalog.load(train_path))
val = _concat_chunks(context.catalog.load(val_path))
test = _concat_chunks(context.catalog.load(test_path))

In [ ]:
def _status_change(x: pd.Series) -> str:
    """Based on difference of the following rows create label which indicates
    if given product was added, dropped or maintained in comparison with last 
    month for given customer.

    Args:
        x (pd.DataFrame): imputed santander train dataframe

    Returns:
        str: target label - added/dropped/maintained
    """
    diffs = x.diff().fillna(0)
    # First occurrence is considered as "Maintained"
    label = ["Added" if i == 1 \
         else "Dropped" if i == -1 \
         else "Maintained" for i in diffs]
    return label


def _target_processing_santander(train_df: Iterator[pd.DataFrame],
                                val_df: Iterator[pd.DataFrame]) -> Tuple:
    """Preprocess target columns to focus on products that will be bought
    in the next month

    Args:
        input_train_df (Iterator[pd.DataFrame]): imputed santander train
        dataframe
        input_val_df (Iterator[pd.DataFrame]): imputed santander validation 
        dataframe

    Returns:
        Tuple: processed train and validation dataframes
    """
    train_len = len(train_df)
    df = pd.concat([train_df, val_df])
    feature_cols = df.iloc[:1,].filter(regex="ind_+.*ult.*").columns.values
    df = df.sort_values(['ncodpers', 'fecha_dato']).reset_index(drop=True)
    # Apply status change labeling
    df.loc[:, feature_cols] = (df.loc[:, [i for i in feature_cols]
                               + ["ncodpers"]].groupby("ncodpers")
                               .transform(_status_change))
                               
    df = df.sort_values(['fecha_dato']).reset_index(drop=True)

    train_df = df.iloc[:train_len, :]
    val_df = df.iloc[train_len:, :]
    return (train_df, val_df)

In [ ]:
transactions_train_1, transactions_val_1 = _target_processing_santander(train, val)

In [ ]:
transactions_train_1

In [ ]:
transactions_train_1.loc[transactions_train_1.loc[:, 'ind_cco_fin_ult1'] == 'Added'].sort_values('ncodpers').head()

In [ ]:
train.loc[train.ncodpers == 360274, :]

In [ ]:
def _status_change_faster(x: pd.Series) -> str:
    """Based on difference of the following rows create label which indicates
    if given product was added, dropped or maintained in comparison with last 
    month for given customer.

    Args:
        x (pd.DataFrame): imputed santander train dataframe

    Returns:
        str: target label - added/dropped/maintained
    """
    # First occurrence is considered as "Maintained"
    label = ["Added" if i == 1 \
         else "Dropped" if i == -1 \
         else "Maintained" for i in x]
    return label


def _target_processing_santander_faster(train_df: Iterator[pd.DataFrame],
                                val_df: Iterator[pd.DataFrame]) -> Tuple:
    """Preprocess target columns to focus on products that will be bought
    in the next month

    Args:
        input_train_df (Iterator[pd.DataFrame]): imputed santander train
        dataframe
        input_val_df (Iterator[pd.DataFrame]): imputed santander validation 
        dataframe

    Returns:
        Tuple: processed train and validation dataframes
    """
    train_len = len(train_df)
    df = pd.concat([train_df, val_df])
    feature_cols = df.iloc[:1,].filter(regex="ind_+.*ult.*").columns.values
    df = df.sort_values(['ncodpers', 'fecha_dato']).reset_index(drop=True)
    # Apply status change labeling
    s = df.loc[:, [i for i in feature_cols] + ["ncodpers"]]
    df.loc[:, feature_cols] = (s.loc[:, [i for i in feature_cols]].diff()
                               .where(s.duplicated(["ncodpers"],
                               keep='first'))
                               ).fillna(0).transform(_status_change_faster)
    df = df.sort_values(['fecha_dato']).reset_index(drop=True)

    train_df = df.iloc[:train_len, :]
    val_df = df.iloc[train_len:, :]
    return (train_df, val_df)

In [ ]:
transactions_train_2, transactions_val_2 = _target_processing_santander_faster(train, val)

In [ ]:
transactions_train_2.loc[transactions_train_2.loc[:, 'ind_cco_fin_ult1'] == 'Added'].sort_values('ncodpers').head()

In [ ]:
train.loc[train.ncodpers == 348324, :]

In [ ]:
transactions_train_2.equals(transactions_train_1)
transactions_val_2.equals(transactions_val_1)